In [87]:
import pandas as pd
from progressbar import ProgressBar

data_dir = "../data/"

## Ingest and format data

In [6]:
# column_names_list = ["station_id", 'date', 'name', 'value', 'range_flag', 'door_flag', 'frozen_flag', 'manual_flag']

# # Ingest raw data for single soil station.
# df = pd.read_csv(
#     data_dir+"acclima_soil_water_rleeper_FULL.csv",
#     header="infer",
#     index_col=False
# )

# df = df.drop(columns=["TAGS"])
# df.columns = column_names_list

# df.head()

In [76]:
new_station_ids = [
    int(stat_id)
    if stat_id!="UN" else stat_id
    for stat_id in df.station_id.values
]

In [79]:
df.station_id = new_station_ids

In [80]:
# Create an array of all unique station IDs.
station_id_array = df.station_id.unique()

In [97]:
# Isolate the current station's rows.
df_station = df[df.station_id == station_id_array[0]]

# Convert all values in "date" column to datetime objects.
df_station["date"] = pd.to_datetime(df_station["date"], format='%Y-%m-%d %H:%M:%S',)

# Pivot the station's data and target values into ML-compatible arrays. 
df_station_data =\
    df_station.pivot(index="date", columns="name", values="value")

# Create the class labels DF for the station.
df_station_targets =\
    df_station.pivot(
        index="date", columns="name", values="manual_flag"
    )

# Create a precipitation and temperature DF.
df_station_pt_data = df_station_data[["p_official", "t_official"]]

# Drop irrelevant columns from the DFs.
df_station_data = df_station_data.drop(
        columns=["p_official", "t_official"]
    )
df_station_targets = df_station_targets.drop(
        columns=["p_official", "t_official"]
    )

# Get rid of rows of missing data and then drop those rows from the targets dataframe.
df_station_data = df_station_data.dropna(how="all")
df_station_pt_data = df_station_pt_data.loc[df_station_data.index]
df_station_targets = df_station_targets.loc[df_station_data.index]

# Add the precip and temp data back onto the station data.
df_station_data["p_official"] = df_station_pt_data["p_official"]
df_station_data["t_official"] = df_station_pt_data["t_official"]

# # Save the station's data and labels as a CSV
df_station_data.to_csv("../data/stations/data_"+str(station_id_array[0])+".csv")
df_station_targets.to_csv("../data/stations/targets_"+str(station_id_array[0])+".csv")

/Users/garrettgraham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [107]:
df_station_data["p_official"] = df_station_pt_data["p_official"]
df_station_data["t_official"] = df_station_pt_data["t_official"]

## Subset and cache all station data

In [110]:
pbar = ProgressBar()

for station_id in pbar(station_id_array):
    
    # Isolate the current station's rows.
    df_station = df[df.station_id == station_id]

    # Convert all values in "date" column to datetime objects.
    df_station["date"] = pd.to_datetime(df_station["date"], format='%Y-%m-%d %H:%M:%S',)

    # Pivot the station's data and target values into ML-compatible arrays. 
    df_station_data =\
        df_station.pivot(index="date", columns="name", values="value")

    # Create the class labels DF for the station.
    df_station_targets =\
        df_station.pivot(
            index="date", columns="name", values="manual_flag"
        )

    # Create a precipitation and temperature DF.
    df_station_pt_data = df_station_data[["p_official", "t_official"]]

    # Drop irrelevant columns from the DFs.
    df_station_data = df_station_data.drop(
            columns=["p_official", "t_official"]
        )
    df_station_targets = df_station_targets.drop(
            columns=["p_official", "t_official"]
        )

    # Get rid of rows of missing data and then drop those rows from the targets dataframe.
    df_station_data = df_station_data.dropna(how="all")
    df_station_pt_data = df_station_pt_data.loc[df_station_data.index]
    df_station_targets = df_station_targets.loc[df_station_data.index]

    # Add the precip and temp data back onto the station data.
    df_station_data["p_official"] = df_station_pt_data["p_official"]
    df_station_data["t_official"] = df_station_pt_data["t_official"]

    # # Save the station's data and labels as a CSV
    df_station_data.to_csv("../data/stations/data_"+str(station_id)+".csv")
    df_station_targets.to_csv("../data/stations/targets_"+str(station_id)+".csv")

/Users/garrettgraham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
100% |########################################################################|


## Verify that data and targets were properly cached

In [135]:
# Check whether anomalies were actually recorded for at least some stations.
data_dir = "../data/stations/"
for test_station_id in station_id_array:
    test_data = pd.read_csv(data_dir+f"data_{test_station_id}.csv")
    test_targets = pd.read_csv(data_dir+f"targets_{test_station_id}.csv")

    tt = test_targets.drop(columns=["date"])
    anom_sum = (tt > 0).sum().sum()
    if anom_sum > 0:
        print(test_station_id)

3728
3758
4126
4141
4236
4990
4994
13301
23906
26563
53152
53155
53877
53968
54808
54811
54854
63826
63828
63829
63831
63850
63855
63858
63869
64756
92827
93243
94077
94995
UN


It's all looking pretty reasonable! Time to do some data exploration on it.